<a href="https://colab.research.google.com/github/brahma2024/Deep-Implicit-Layers/blob/main/ch1_2_Deep_Implicit_Layer_Fixedpoint_through_Implicit_Differentiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving for Fixed Point Layer using Implicit Differentiation

- Reference https://implicit-layers-tutorial.org/introduction/
- section: **Differentiation in Implicit Layers**

Note:
- function g can be used to represent convolutions, self-attention, or any other feature

# Key Points
1. Solve for the root of the implicit layer g(x, z*) = 0, outisde of the automatic diferentiation tape
2. Use the automatic differentiation tape to run the following assignment within the differentiation tape:
    z:=z⋆−g(x,z⋆)
  - this is like reinserting the partial dervatives −∂g/∂x to the autograd tape
3. Add a backward hook to the backward pass that multiplies by (∂g/∂z⋆).transpose
  - this will fix the backward pass so that it correctly implements the gradient according to the implicit function theorem

In [1]:
import torch
import torch.nn as nn
from torch.profiler import profile, record_function, ProfilerActivity
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [18]:
class TanhNewtonImplicitLayer(nn.Module):
  def __init__(self, out_features, tol=1e-4, max_iter=1):
    super().__init__()
    self.linear = nn.Linear(out_features, out_features, bias=False) # (C, C)
    self.tol = tol
    self.max_iter = max_iter

  def forward(self, x):
    # Run Newton's method outside of the autograd framework
    with torch.no_grad():
      z = torch.tanh(x) # (B, C) | Initial guess, z0 = tanh(x)
      print(f'0. Start with :{z.shape=}')
      self.iterations = 0
      while self.iterations < self.max_iter: # by iteratively doing it cond: iterations < max_iter or (z - tanh(z_linear) < self.tol | objective is to find J = (∂g/∂z⋆)
        z_linear = self.linear(z) + x # (B, C)
        g = z - torch.tanh(z_linear) # (B, C)
        self.err = torch.norm(g) # L2 norm | euclidean distance
        if self.err < self.tol:
          break

        # newton Root Finding Method: 1. compute the gradient
        # Jacobian = (∂g/∂z⋆), where
        # g(x,z)= z − tanh(Wz+x), hence
        # J = I - diag(tanh'(Wz* + x)) * W, where
        # Identity matrix = I
        # Adjustment term = ∂(tanh(Wz*+x))/∂z = diag(tanh'(Wz* + x)) * W
        identity_matrix = torch.diag(torch.ones(g.shape[1]))         # (C, C) | square identity matriX
        print(f'1 {identity_matrix.shape=}')
        identity_matrix = identity_matrix.unsqueeze(0)               # (1, C, C) | add a batch dimension to I for broadcasting
        print(f'2 {identity_matrix.shape=}')
        adjustment_term_1 = (1/torch.cosh(z_linear)**2).unsqueeze(2) # (B, C, 1) | add 1 dim for channels
        print(f'3 {adjustment_term_1.shape=}')
        adjustment_term_2 = self.linear.weight.unsqueeze(0)          # (1, C, C) | add 1 dim for C
        print(f'4 {adjustment_term_2.shape=}')
        adjustment_term = adjustment_term_1 * adjustment_term_2      # (B, C, C)
        print(f'5 {adjustment_term.shape=}')

        J = identity_matrix - adjustment_term # (B, C, C) | (∂g/∂z⋆)
        print(f'6 {J.shape=}')
        # Newton Root finding method: 2. newton step update
        # compute delta_z
        delta_z = torch.linalg.solve(J, g.unsqueeze(2)) # (B, C, 1)
        print(f'7 {delta_z.shape=}')
        z = z - delta_z.squeeze(2) # (B, C) | z-new = z − (∂g/∂z⋆)**−1 * g(z)
        print(f'8 {z.shape=}')
        print(f'{z[0]=} | {self.err=}')
        self.iterations += 1


    # renengage autograd and add the gradient hook
    z = torch.tanh(self.linear(z) + x) # Recompute z to include it in the autograd computation graph
    print(f'9 {z.shape=}')
    # jacobian_transpose = J.transpose(1, 2)
    # Solving the linear system 𝐽.𝑇ranspose * 𝛿 = grad, where 𝐽.𝑇ranspose is the transposed Jacobian and grad is the incoming gradient.
    # call register_hook on a tensor, you are effectively saying,
    # Whenever a gradient is computed for this tensor during backpropagation, apply this function to the gradient before proceeding.
    # grad in register_hook is the gradient of the loss wrt to the tensor z
    # It is computed during the backward pass.
    # Specifically, it is the derivative of the loss function with respect to z, which is propagated from the loss through the layers of the network back to z.
    z.register_hook(lambda grad: torch.linalg.solve(J.transpose(1, 2), g.unsqueeze(2)).squeeze(2))
    print(f'10 {z.shape=} \n')
    return z


#Summing Up: Forward Pass
- Objective: Find Jacobian matrix (∂g/∂z⋆)
- Step 1: start with an initiat guess for z = tanh(x)
- Next Steps to be executed iteratively
  - Step 2:
    - 'self.linear(z) + x': Applies a linear transformation to z and adds x.
    - 'torch.tanh(...)': Applies the hyperbolic tangent activation function.
  - Step 3: Calculate the jacobian matrix (gradient of function g(x, z*) wrt to
   intermediate output z*
   - J = I - tanh'(WZ* + X)
   - Some Notes about the Jacobian calculation:
    - g(z*, x) is of size (B, C), i.e. each batch sample is a vector of dimension/channels/features = C
    - First we calculate the element-wise derivative of each element in each batch, i.e. the derivative of an element in a batch sample wrt to other elements will be zero
    - this will result in a final jacobian of size (B, C, C) which will be block-diagonal
    - The terms computed in order to compute the final jacobian matrix, include
      - Identity matrix is of size (C, C)
      - sehc(z)^2 = (1/cosh(z)^2) | this will be size (B, C) since we are calculating element-wise derivative
      - next, multiple the sec^2 derivative term with the weights of the linear layer W which is of size (C, C)
      - in order to broadcast derivative (sech^2) for each element to be multiplied with each respective element of linear layer weights, we do the following transformation
      - sech^2 [:, :, None] | W [None, :, :] | i.e. add one spurious dimension to convert (B, C) -> (B, C, 1) | add one spurious dimension to convert (C, C) -> (1, C, C)
      - resulting in a jacobian matrix of size (B, C, C) which stores the final gradients of each element in channel-dimention for each vector in batch (batch sample)


# Summing up: Backward Pass
- Once the gradients (Jacobian Matrix) = (∂g/∂z⋆) is computed
- we recompute z = tanh(Wz + x) | creates the computational graph
- register a grad hook to z which will propagate the gradients in the backward pass


In [17]:
from torch.autograd import gradcheck

layer = TanhNewtonImplicitLayer(5, tol=1e-10).double()
gradcheck(layer, torch.randn(3, 5, requires_grad=True, dtype=torch.double), check_undefined_grad=False)

0. Start with :z.shape=torch.Size([3, 5])
1 identity_matrix.shape=torch.Size([5, 5])
2 identity_matrix.shape=torch.Size([1, 5, 5])
3 adjustment_term_1.shape=torch.Size([3, 5, 1])
4 adjustment_term_2.shape=torch.Size([1, 5, 5])
5 adjustment_term.shape=torch.Size([3, 5, 5])
6 J.shape=torch.Size([3, 5, 5])
7 delta_z.shape=torch.Size([3, 5, 1])
8 z.shape=torch.Size([3, 5])
z[0]=tensor([-0.3671, -0.8310, -0.5719,  0.3761,  0.8552], dtype=torch.float64) | self.err=tensor(0.8252, dtype=torch.float64)
9 z.shape=torch.Size([3, 5])
10 z.shape=torch.Size([3, 5])
0. Start with :z.shape=torch.Size([3, 5])
1 identity_matrix.shape=torch.Size([5, 5])
2 identity_matrix.shape=torch.Size([1, 5, 5])
3 adjustment_term_1.shape=torch.Size([3, 5, 1])
4 adjustment_term_2.shape=torch.Size([1, 5, 5])
5 adjustment_term.shape=torch.Size([3, 5, 5])
6 J.shape=torch.Size([3, 5, 5])
7 delta_z.shape=torch.Size([3, 5, 1])
8 z.shape=torch.Size([3, 5])
z[0]=tensor([-0.3671, -0.8310, -0.5719,  0.3761,  0.8552], dtype=torch

GradcheckError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[ 0.7941, -0.0164,  0.2345, -0.2156, -0.0710,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0374,  0.2749, -0.0225, -0.0118,  0.0019,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.1351, -0.0235,  0.7250,  0.1227,  0.0316,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0341, -0.0669, -0.1918,  0.7841, -0.0254,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0234,  0.0052, -0.0846,  0.0446,  0.2596,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.8772, -0.1005,  0.2043,
         -0.0248, -0.1908,  0.0000,  0.0000,  0.0000,  0.0000,  0.0

In [ ]:
g = torch.randn(10, 50)
im1 = torch.diag(torch.ones(g.shape[1]))
im1 = im1.unsqueeze(0)
td1 = 1/(torch.cosh(g)**2)
td1 = td1.unsqueeze(2)
im2 = torch.eye(g.shape[1])[None, :, :]
td2 = (1/(torch.cosh(g)**2))[:, :, None]
im1.shape, im2.shape, torch.allclose(im1, im2), td1.shape, td2.shape, torch.allclose(td1, td2)

j1 = im1 - td1
j1.shape, g.shape

delta_z = torch.linalg.solve(j1, g.unsqueeze(2))
delta_z.shape